In [1]:
import pickle
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")
#user_data=pd.read_csv('Dataset/Users.csv',header=[0])
book_data=pd.read_csv('Dataset/Books.csv',header=[0])
#ratings_data=pd.read_csv('Dataset/Ratings.csv',header=[0])
book_data.describe()
#book_data.describe()
#ratings_data.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [6]:
#print(book_data.head)
#DATA CLEANING

book_data.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
book_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [7]:
print("Columns: ", list(book_data.columns))
book_data.head()

Columns:  ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [8]:
book_data.isnull().sum() 

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
dtype: int64

In [9]:
book_data.loc[book_data['Book-Author'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [10]:
book_data.loc[book_data['Publisher'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [11]:
book_data.at[187689 ,'Book-Author'] = 'Other'
book_data.at[128890 ,'Publisher'] = 'Other'
book_data.at[129037 ,'Publisher'] = 'Other'

In [12]:
print(book_data.at[187689,'Book-Author'])

Other


In [13]:
book_data['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [14]:
#pd.set_option('display.max_colwidth', -1)
book_data.loc[book_data['Year-Of-Publication'] == 'DK Publishing Inc',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [15]:
book_data.loc[book_data['Year-Of-Publication'] == 'Gallimard',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...


In [16]:
book_data.at[209538 ,'Publisher'] = 'DK Publishing Inc'
book_data.at[209538 ,'Year-Of-Publication'] = 2000
book_data.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
book_data.at[209538 ,'Book-Author'] = 'Michael Teitelbaum'

book_data.at[221678 ,'Publisher'] = 'DK Publishing Inc'
book_data.at[221678 ,'Year-Of-Publication'] = 2000
book_data.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
book_data.at[209538 ,'Book-Author'] = 'James Buckley'

book_data.at[220731 ,'Publisher'] = 'Gallimard'
book_data.at[220731 ,'Year-Of-Publication'] = '2003'
book_data.at[209538 ,'Book-Title'] = 'Peuple du ciel - Suivi de Les bergers '
book_data.at[209538 ,'Book-Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [17]:
book_data['Year-Of-Publication'] = book_data['Year-Of-Publication'].astype(int)

In [18]:
print(sorted(list(book_data['Year-Of-Publication'].unique())))

[0, 1376, 1378, 1806, 1897, 1900, 1901, 1902, 1904, 1906, 1908, 1909, 1910, 1911, 1914, 1917, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2010, 2011, 2012, 2020, 2021, 2024, 2026, 2030, 2037, 2038, 2050]


In [19]:
book_data.loc[book_data['Year-Of-Publication'] > 2021, 'Year-Of-Publication'] = 2002
book_data.loc[book_data['Year-Of-Publication'] == 0, 'Year-Of-Publication'] = 2002

In [20]:
book_data['ISBN'] = book_data['ISBN'].str.upper()

In [21]:
book_data.drop_duplicates(keep='last', inplace=True) 
book_data.reset_index(drop = True, inplace = True)

In [22]:
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271047 entries, 0 to 271046
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271047 non-null  object
 1   Book-Title           271047 non-null  object
 2   Book-Author          271047 non-null  object
 3   Year-Of-Publication  271047 non-null  int32 
 4   Publisher            271047 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.3+ MB


In [23]:
#userdataset_preprocessing
users=pd.read_csv('Dataset/Users.csv',header=[0])

In [24]:
print("Columns: ", list(users.columns))
users.head()

Columns:  ['User-ID', 'Location', 'Age']


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [25]:
## Checking null values
print(users.isna().sum())

User-ID          0
Location         0
Age         110762
dtype: int64


In [26]:
users.drop(['Location'], axis=1, inplace=True)
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   User-ID  278858 non-null  int64  
 1   Age      168096 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 4.3 MB
None


In [27]:
users.drop_duplicates(keep='last', inplace=True)
users.reset_index(drop=True, inplace=True)
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   User-ID  278858 non-null  int64  
 1   Age      168096 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 4.3 MB
None


In [28]:
#Ratings dataset cleaning

ratings=pd.read_csv('Dataset/Ratings.csv',header=[0])

In [29]:
print("Columns: ", list(ratings.columns))
ratings.info()

Columns:  ['User-ID', 'ISBN', 'Book-Rating']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [30]:
ratings.isnull().sum() 

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [31]:
ratings['ISBN'] = ratings['ISBN'].str.upper()

In [32]:
ratings.drop_duplicates(keep='last', inplace=True)
ratings.reset_index(drop=True, inplace=True)

In [33]:
ratings.info()
users.info()
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   User-ID  278858 non-null  int64  
 1   Age      168096 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 4.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271047 entries, 0 to 271046
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271047 non-null  object
 1   Book-Title           271047 non-null  obj

In [34]:
dataset = pd.merge(book_data, ratings, on='ISBN', how='inner')
dataset = pd.merge(dataset, users, on='User-ID', how='inner')
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031273 entries, 0 to 1031272
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ISBN                 1031273 non-null  object 
 1   Book-Title           1031273 non-null  object 
 2   Book-Author          1031273 non-null  object 
 3   Year-Of-Publication  1031273 non-null  int32  
 4   Publisher            1031273 non-null  object 
 5   User-ID              1031273 non-null  int64  
 6   Book-Rating          1031273 non-null  int64  
 7   Age                  753417 non-null   float64
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 66.9+ MB


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,0,NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,NaN


In [35]:
dataset1 = dataset[dataset['Book-Rating'] != 0]
dataset1 = dataset1.reset_index(drop = True)
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383892 entries, 0 to 383891
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 383892 non-null  object 
 1   Book-Title           383892 non-null  object 
 2   Book-Author          383892 non-null  object 
 3   Year-Of-Publication  383892 non-null  int32  
 4   Publisher            383892 non-null  object 
 5   User-ID              383892 non-null  int64  
 6   Book-Rating          383892 non-null  int64  
 7   Age                  269663 non-null  float64
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 22.0+ MB


In [36]:
dataset2 = dataset[dataset['Book-Rating'] == 0]
dataset2 = dataset2.reset_index(drop = True)
dataset2.shape

(647381, 8)

In [37]:
avg=dataset1.groupby('Book-Title')['Book-Rating'].mean()
avg.head()

Book-Title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)     9.000000
 Ask Lily (Young Women of Faith: Lily Series, Book 5)                                                          8.000000
 Dark Justice                                                                                                 10.000000
 Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth               7.142857
 Final Fantasy Anthology: Official Strategy Guide (Brady Games)                                               10.000000
Name: Book-Rating, dtype: float64

In [61]:
def avgRating(newdf, df):
    newdf['Average Rating'] = 0
    for x in range(len(newdf)):
        l = list(df.loc[df['Book-Title'] == newdf['Book-Title'][x]]['Book-Rating'])
        newdf['Average Rating'][x] = sum(l)/len(l)
    return newdf

df = pd.DataFrame(dataset1['Book-Title'].value_counts())
df['Total-Ratings'] = df['Book-Title']
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
df = df.drop('index',axis=1)
df = pd.read_pickle('weightedData')

In [62]:
## C - Mean vote across the whole
C = df['Average Rating'].mean()

## Minimum number of ratings required to be in the chart
m = df['Total-Ratings'].quantile(0.90)

In [65]:
print(C)
print(m)

7.407699682867468
5.0


In [63]:
def weighted_rating(x, m=m, C=C): 
    v = x['Total-Ratings']    #v - number of votes
    R = x['Average Rating']   #R - Average Rating   
    return (v/(v+m) * R) + (m/(m+v) * C)

In [64]:
df = df.loc[df['Total-Ratings'] >= m]

df['score'] = df.apply(weighted_rating, axis=1)
df = df.sort_values('score', ascending=False)

print("Recommended Books:-\n")
df.head(5)

Recommended Books:-



,Book-Title,Total-Ratings,Average Rating,score
4794,Postmarked Yesteryear: 30 Rare Holiday Postcards,11,10,9.189906
7272,The Sneetches and Other Stories,8,10,9.002961
17,Harry Potter and the Prisoner of Azkaban (Book 3),277,9,8.971768
28,Harry Potter and the Goblet of Fire (Book 4),247,9,8.968407
42,Harry Potter and the Order of the Phoenix (Boo...,211,9,8.963141


In [42]:
#RECOMMENDATION BASED ON AVERAGE RATING

#print(dataset1.info())
#print(df['Book-Title'][1])
#print(dataset1.at[dataset1.loc[dataset1['Book-Title'] == df['Book-Title'][0]],['Book-Rating']])
#l=list(dataset1.loc[dataset1['Book-Title'] == df['Book-Title'][0]]['Book-Rating'])
#df.at[0,'Average Rating'] = sum(l)/len(l)
#print(df.at[0,'Average Rating'])

def avgRating(newdf, df):
    for x in range(10):
        l = list(df.loc[df['Book-Title'] == newdf['Book-Title'][x]]['Book-Rating'])
        #newdf['Average Rating'][x] = sum(l)/len(l)
        newdf.at[x,'Average Rating'] = sum(l)/len(l)
    return newdf

df = pd.DataFrame(dataset1['Book-Title'].value_counts()) # returns object containing counts of unique values
#print(df.head())
df['Total-Ratings'] = df['Book-Title']
#print(df.index)
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
#print(df.head())
df = df.drop('index',axis=1)
df['Average Rating'] = 0

df = avgRating(df, dataset1)
df=df.sort_values('Average Rating',ascending=False)
print(df.head(5))
#print(df['Book-Title'][0:5])

                                         Book-Title  Total-Ratings  \
9  Harry Potter and the Chamber of Secrets (Book 2)            326   
3                           The Secret Life of Bees            406   
2                                 The Da Vinci Code            494   
0                         The Lovely Bones: A Novel            707   
5               The Red Tent (Bestselling Backlist)            383   

   Average Rating  
9        8.840491  
3        8.477833  
2        8.439271  
0        8.185290  
5        8.182768  


In [43]:
#Book By Same Author and Same Publisher of given book

def printBook(k, n):
    z = k['Book-Title'].unique()
    for x in range(len(z)):
        print(z[x])
        if x >= n-1:
            break
def get_books(dataframe, name, n):
    print("\nBooks by same Author:\n")
    d = dataframe[dataframe['Book-Title'] == bookName]
    #print(d)
    au = d['Book-Author'].unique()
    print(au)
    data = dataset1[dataset1['Book-Title'] != name]

    if au[0] in list(data['Book-Author'].unique()):
        k2 = data[data['Book-Author'] == au[0]]
    k2 = k2.sort_values(by=['Book-Rating'])
    printBook(k2, n)

    print("\n\nBooks by same Publisher:\n")
    au = d['Publisher'].unique()
    print(au)
    if au[0] in list(data['Publisher'].unique()):
        k2 = pd.DataFrame(data[data['Publisher'] == au[0]])
    k2=k2.sort_values(by=['Book-Rating']) 
    printBook(k2, n)
bookName=input("Enter book name") # Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
number=int(input("Enter number of books to be recommended"))

if bookName in list(dataset1['Book-Title'].unique()):
    get_books(dataset1, bookName, number)
else:
    print("Invalid Book Name!!")

Enter book nameHarry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Enter number of books to be recommended5

Books by same Author:

['J. K. Rowling']
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter y el cÃ¡liz de fuego
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Sorcerer's Stone (Book 1)


Books by same Publisher:

['Arthur A. Levine Books']
The Seeing Stone
The Slightly True Story of Cedar B. Hartley: Who Planned to Live an Unusual Life
Harry Potter and the Chamber of Secrets (Harry Potter)
The Story of the Seagull and the Cat Who Taught Her To Fly
Book! Book! Book!


In [44]:
#Book By Same Author and Same Publisher

def printBook(k, n):
    z = k['Book-Title'].unique()
    for x in range(len(z)):
        print(z[x])
        if x >= n-1:
            break
def get_Author_books(dataframe, bookAuthorName, n):
    print("\nBooks by same Author:\n")
    
    k2 = dataframe[dataframe['Book-Author'] == bookAuthorName]
    k2 = k2.sort_values(by=['Book-Rating'])
    printBook(k2, n)
    
def get_Publisher_books(dataframe, bookPublisherName, n):
    print("\n\nBooks by same Publisher:\n")
    
    k2 = dataframe[dataframe['Publisher'] == bookPublisherName]
    k2=k2.sort_values(by=['Book-Rating']) 
    printBook(k2, n)
inputName=input("Enter book Author or Publisher name") # Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
#bookPublisherName=input("Enter book Publisher name")
number=int(input("Enter number of books to be recommended"))

if inputName in list(dataset1['Book-Author'].unique()):
    get_Author_books(dataset1, inputName, number)
elif inputName in list(dataset1['Publisher'].unique()):
    get_Publisher_books(dataset1, inputName, number)
else:
    print("Invalid Book Author or Publisher Name!!")

Enter book Author or Publisher nameHarry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Enter number of books to be recommended5
Invalid Book Author or Publisher Name!!


In [44]:
#claculates total-ratings each book got and creates a new dataframe having columns bt,isbn,ba,ui,br,tr
df = pd.DataFrame(dataset1['Book-Title'].value_counts())
df['Total-Ratings'] = df['Book-Title']
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
df = df.drop('index',axis=1)
df = dataset1.merge(df, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')
df = df.drop(['Year-Of-Publication','Publisher','Age'], axis=1)
print(df.head(10))

         ISBN                                         Book-Title  \
0  0002005018                                       Clara Callan   
1  074322678X            Where You'll Find Me: And Other Stories   
2  0887841740                                 The Middle Stories   
3  1552041778                                           Jane Doe   
4  1567407781       The Witchfinder (Amos Walker Mystery Series)   
5  1575663937  More Cunning Than Man: A Social History of Rat...   
6  1881320189                      Goodbye to the Buttermilk Sky   
7  0786868716                 The Five People You Meet in Heaven   
8  0151008116                                         Life of Pi   
9  0312195516                The Red Tent (Bestselling Backlist)   

            Book-Author  User-ID  Book-Rating  Total-Ratings  
0  Richard Bruce Wright        8            5              9  
1           Ann Beattie        8            5              1  
2           Sheila Heti        8            5              1  

In [45]:
dataset1.loc[dataset1['ISBN'] == '1881320189',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
6,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,8,7,NaN
11371,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,226745,7,38.0


In [46]:
dataset1.loc[dataset1['ISBN'] == '0786868716',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
7,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,49.0
45,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11676,9,NaN
7777,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,263325,8,27.0
7890,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,110912,10,36.0
11560,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,7346,7,49.0
...,...,...,...,...,...,...,...,...
174386,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,263560,10,36.0
174398,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,267117,10,57.0
174401,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,269340,9,44.0
174408,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,269702,6,52.0


In [47]:
dataset1.loc[dataset1['ISBN'] == '0151008116',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
8,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,49.0
83,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11676,6,NaN
9587,0151008116,Life of Pi,Yann Martel,2002,Harcourt,113519,9,NaN
12374,0151008116,Life of Pi,Yann Martel,2002,Harcourt,16795,10,47.0
20743,0151008116,Life of Pi,Yann Martel,2002,Harcourt,118659,8,37.0
...,...,...,...,...,...,...,...,...
212245,0151008116,Life of Pi,Yann Martel,2002,Harcourt,256651,10,NaN
212252,0151008116,Life of Pi,Yann Martel,2002,Harcourt,262769,7,33.0
212254,0151008116,Life of Pi,Yann Martel,2002,Harcourt,268690,9,NaN
212255,0151008116,Life of Pi,Yann Martel,2002,Harcourt,270234,7,NaN


In [3]:
popularity_threshold = 80
popular_book = df[df['Total-Ratings'] >= popularity_threshold]
popular_book = popular_book.reset_index(drop = True)
#print(popular_book)

NameError: name 'df' is not defined

In [134]:

testdf = pd.DataFrame()
testdf['ISBN'] = popular_book['ISBN']
testdf['Book-Rating'] = popular_book['Book-Rating']
testdf['User-ID'] = popular_book['User-ID']
testdf = testdf[['User-ID','Book-Rating']].groupby(testdf['ISBN'])
print(testdf.first())
#print(testdf.groups) #have the data of users who rated each individual book
#print(dataset1.info())

            User-ID  Book-Rating
ISBN                            
0001047973    11944            9
0002558122    11676            8
000649840X    11676            8
002542730X    11676            6
0048231398    12272           10
...             ...          ...
B0000T6KHI   112001           10
B0000T6KIM   204359            4
B0000VZEH8   241791            9
B0001FZGBC    58551            7
B0001FZGPI   222207            7

[1871 rows x 2 columns]


In [51]:
testdf = pd.DataFrame()
testdf['ISBN'] = popular_book['ISBN']
testdf['Book-Rating'] = popular_book['Book-Rating']
testdf['User-ID'] = popular_book['User-ID']
testdf = testdf[['User-ID','Book-Rating']].groupby(testdf['ISBN'])
print(testdf.groups)

{'0001047973': [7259, 7619], '0002558122': [565, 25268, 30245, 48893, 56436, 60941, 63777, 64811, 64953, 64954], '000649840X': [570, 5540, 7007, 7926, 9179, 9231, 9441, 9553, 9621, 11203, 11479, 13842, 14325, 14818, 18536, 23370, 24993, 31513, 33303, 37890, 38827, 43176, 48196, 51580, 54538, 56736, 58479, 60423, 60711, 61067, 61345, 62505, 62599, 63747, 64917, 64990, 64991, 64992, 64993, 64994, 64995, 64996, 64997], '002542730X': [359, 945, 2043, 3860, 4130, 5092, 5536, 6967, 6993, 10271, 12092, 13631, 13820, 13893, 13996, 14234, 14758, 15326, 16380, 16560, 16840, 17231, 17823, 18454, 18811, 19017, 20662, 20859, 22686, 22735, 23492, 23743, 23799, 24517, 24975, 24991, 25873, 25927, 26399, 27942, 32115, 33033, 34485, 36369, 38510, 38876, 41676, 41886, 43174, 48044, 48465, 48943, 50512, 51184, 52838, 53321, 55766, 56379, 56598, 57060, 58882, 59091, 59097, 59660, 61034, 61085, 62361, 62362, 62363, 62364, 62365, 62366, 62368, 62369, 62370, 62372, 62374], '0048231398': [7270, 17782, 27944, 6

In [52]:
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    #groupDF= groupDF.reset_index(drop = True)
    print(groupDF)
    
    #print(groupDF.iloc[0,0],groupDF.iloc[0,1])
    #Dataframe.iloc[] method is used when the index label of a data frame is something other than numeric series of 0, 1, 2, 3….n

      User-ID  Book-Rating
7259    11944            9
7619    23902            9
       User-ID  Book-Rating
565      11676            8
25268   202664            8
30245    81274            8
48893   191521            9
56436    71837           10
60941   201537            8
63777   196118            8
64811   236173            9
64953   131398            9
64954   216420            5
       User-ID  Book-Rating
570      11676            8
5540     86243            8
7007       901            9
7926     32440            8
9179     69405            5
9231     70052            3
9441     76350            7
9553     81227            9
9621     81855            8
11203   140000            6
11479   147156            8
13842   231613            8
14325   246678            2
14818   262407            7
18536   120080           10
23370   149908            8
24993   164096            7
31513   148258            7
33303   114279            9
37890   256820           10
38827   221672         

[94 rows x 2 columns]
       User-ID  Book-Rating
602      11676           10
646      85526            7
685     200273            5
900      46398            5
3212    129084            8
3700    160819            5
3916    189655            5
6412    204864            9
6687    271135            5
8658     52917            8
11892   168064            8
13283   217386            7
14102   240403            4
14543   252820            9
14653   255218           10
19306   242083            7
20000   138777            8
20403   239584            8
22793   245645            9
23129    37311            9
23323   138097            8
23759    64333            9
23774   139467            8
25112   101731            9
26560   143302            7
30627   193750            8
31190   244994            8
32129   208605            7
34549    86189            8
35136   135609            9
36578   153054            8
38471    25409            9
39164    46099            8
40364   122782            

64715    73411            7
       User-ID  Book-Rating
73       11676            5
2026     52584           10
2491     97721            6
2621    105979            7
3288    135149            8
...        ...          ...
46796   249637            9
46797   250471            7
46798   250748            7
46800   259265            8
46804   271857            6

[165 rows x 2 columns]
       User-ID  Book-Rating
8896     57039            5
11302   141958            9
11485   147803            8
14133   242006            9
26132   238652            9
35534   278554            8
45884   130705            8
51555   123625            7
60939   171271            8
61079   264111            9
63044     7155           10
63045   104198           10
63046   253613            8
       User-ID  Book-Rating
2349     78973            6
7603     23902            9
11108   137190            7
38782    20032           10
47134   137816            5
55029    10660            8
60421     3799          

       User-ID  Book-Rating
5305     22625            9
5639    100459            8
7376     16999            7
11762   162639           10
17718   237856            6
18921   200226            7
20762   192710            7
22654    48975            8
23266   106225            6
25731   194027            8
27429   225066            8
30063    21618            7
32588   136348            7
36759   201677            5
37635   106331            6
38924   201058            9
39185    55751            5
40804    81324            8
40816    86170            9
41294   239708            8
43283   133258            8
46706    91992            5
51706   231048            7
55144    28899            8
60869    41348            5
61184    20714            5
61185    23665            5
61186    31760            8
61187    31980           10
61188    50301            8
61189    63319           10
61190    72022            7
61191    86203            9
61193   100273            8
61194   129841      

64723   192426           10
       User-ID  Book-Rating
583      11676            9
6499    230249            7
9770     88283           10
9988     95025            8
12149   178199            7
18668   142253            8
38636   112953            8
65120    49626            8
65121   254743            8
       User-ID  Book-Rating
1013    211919            7
1132     92861           10
1143    174304           10
1386     16795            8
1744     31556            5
...        ...          ...
26966   273976            6
26980   274932            7
26981   276165           10
26998   276167            9
27000   276372           10

[243 rows x 2 columns]
       User-ID  Book-Rating
7637     23902           10
18176    66942            5
34779   187065            8
57424   126973            7
64910   102379            9
       User-ID  Book-Rating
89       11676            8
1200       243            9
2174     68256           10
2294     78834           10
3011    123981          

...

65344   183987            5
       User-ID  Book-Rating
558      11676            7
56242    55973            4
61724   189847            8
       User-ID  Book-Rating
7313     15049            9
17826    12538            9
       User-ID  Book-Rating
65435   189232            9
       User-ID  Book-Rating
840      36836           10
41789     3757            9
       User-ID  Book-Rating
33506   134434            8
       User-ID  Book-Rating
13901   234597           10
       User-ID  Book-Rating
473      11676            9
20217   196466            4
21802    65663            7
23253   104880            9
       User-ID  Book-Rating
16669   123867           10
       User-ID  Book-Rating
26294    61854            1
       User-ID  Book-Rating
38805    93034            7
       User-ID  Book-Rating
18563   123883            5
27346   184632           10
28075   138139           10
39824   172119            7
40269   108773           10
42269    51983           10
57699    95907      

40735    58768           10
       User-ID  Book-Rating
25259   189835            5
       User-ID  Book-Rating
43262    85962            8
       User-ID  Book-Rating
62708   169893            8
       User-ID  Book-Rating
48117     3192            6
       User-ID  Book-Rating
7264     11993           10
9024     62891           10
11334   142121            5
64675   166833            8
       User-ID  Book-Rating
34476   185771            8
       User-ID  Book-Rating
149      11676           10
1152    174304            6
3005    123790           10
3228    131837           10
3864    176894            7
5084    269566            7
5440     57105            9
7305     15049            8
7761     29259           10
8703     53418           10
9590     81771            8
10129    95903            9
10413   110029           10
12697   201017           10
12938   206891            7
16304    69078            7
17833    13850            6
19057   220278           10
20649   150979      

10349   109107            7
       User-ID  Book-Rating
436      11676           10
889      46398            5
1157    174304            7
1851     35859            8
1974     40889            9
...        ...          ...
63737    32346           10
63739    47864            6
63740   119271            6
63741   122919            8
63742   166424            7

[70 rows x 2 columns]
       User-ID  Book-Rating
232      11676            9
1245      7346            8
1288      9856            8
2088     55492            7
2417     81298            4
3018    123981            9
4200    216683            8
4258    218121           10
4712    244627           10
5141    273979            9
6516    231210            8
7672     25533            9
8486     44728            9
9846     89207            8
10071    95359           10
10813   126736            6
11638   157127            9
12326   184465            7
13078   209756            6
16140    43359           10
21862    83287           

64931   274152            7
       User-ID  Book-Rating
1136     92861           10
3734    164323            8
3997    197364            8
5208      6242            6
8018     36003            9
8060     37538           10
14786   260183            8
18539   120935            8
19030   210168            8
19284   242083            7
19492   275306           10
19974   130571            6
20457   246311            7
24435   140223            4
24696   220597            7
24732   229313           10
24928   270713            5
25953   165758            5
26873   248477            9
33373   250832            8
34746   148124            7
36460   112298            7
37707   239670            6
41346   247840           10
46763   193612            6
47338     3917            8
47472   273440           10
48282    61759            9
50904    44597           10
51353   209625            6
51659   247537            8
52484   223528            7
57283    81249            9
60008   197345      

64854   258885            8
       User-ID  Book-Rating
9444     76968           10
33537    35433            9
46962   165789           10
64650    36531            6
64684   132790            4
65058   203212            8
65059   207091            8
       User-ID  Book-Rating
39447    84619            4
59620    84867            9
63778    77916            5
64119    13546            7
64120    61986            5
64121    80731            8
64122   119642            4
64123   168327           10
64124   238294            8
       User-ID  Book-Rating
65311    76537            7
       User-ID  Book-Rating
33543   172233            9
33545    34152           10
43628   149510           10
49645   125274           10
61486   209195           10
62075    90839            5
63790   155149            5
63951   222322           10
64093    13039            9
64094   154267            7
64095   179910           10
64096   240137           10
64097   251924            9
64098   273738      

In [53]:
dataset1.loc[dataset1['ISBN'] == '0001047973',:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
44132,0001047973,Brave New World,Aldous Huxley,1999,Trafalgar Square Publishing,11944,9,21.0
46311,0001047973,Brave New World,Aldous Huxley,1999,Trafalgar Square Publishing,23902,9,NaN


In [54]:
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    groupDF= groupDF.reset_index(drop = True)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1]
    print(tempDict)

{11944: 9, 23902: 9}
{11676: 8, 202664: 8, 81274: 8, 191521: 9, 71837: 10, 201537: 8, 196118: 8, 236173: 9, 131398: 9, 216420: 5}
{11676: 8, 86243: 8, 901: 9, 32440: 8, 69405: 5, 70052: 3, 76350: 7, 81227: 9, 81855: 8, 140000: 6, 147156: 8, 231613: 8, 246678: 2, 262407: 7, 120080: 10, 149908: 8, 164096: 7, 148258: 7, 114279: 9, 256820: 10, 221672: 8, 262902: 9, 231854: 6, 28993: 9, 45305: 10, 235948: 7, 147162: 9, 159863: 8, 176638: 8, 126156: 8, 168646: 9, 16383: 10, 43992: 6, 219181: 7, 24922: 9, 39305: 6, 57974: 5, 64171: 10, 73815: 8, 77228: 8, 154112: 9, 234554: 10, 249626: 8}
{11676: 6, 113270: 5, 52584: 10, 175117: 6, 213760: 9, 269566: 7, 86243: 7, 276925: 10, 277427: 10, 104058: 5, 174216: 5, 225763: 7, 229741: 7, 234597: 7, 236399: 10, 244277: 8, 259264: 7, 41084: 7, 80538: 7, 105108: 10, 165582: 5, 235498: 7, 12538: 10, 110934: 6, 173291: 7, 209516: 9, 150979: 8, 10030: 7, 71712: 6, 208671: 7, 183995: 10, 128835: 8, 16996: 8, 152946: 8, 3154: 8, 164096: 8, 174615: 7, 119725:

{11676: 8, 263325: 9, 160819: 10, 260897: 10, 129716: 9, 146932: 10, 37974: 8, 43246: 10, 95025: 8, 246759: 8, 69078: 9, 89602: 9, 201290: 9, 8253: 7, 6575: 8, 45315: 2, 206944: 10, 76499: 10, 162376: 10, 168954: 5, 174914: 9, 273857: 8, 218721: 6, 15184: 8, 20627: 10, 31823: 10, 42978: 9, 60612: 6, 94639: 9, 112148: 9, 117215: 7, 120181: 10, 127203: 4, 147404: 7, 151262: 9, 197502: 9, 203533: 8, 203916: 7, 204802: 7, 209174: 7, 221040: 10}
{85526: 7, 23768: 8, 126492: 10, 144531: 9, 240567: 2, 8570: 8, 187810: 10, 179791: 9, 203017: 8, 246759: 6, 262186: 8, 266865: 10, 266226: 7, 69078: 9, 115435: 10, 179019: 8, 8253: 10, 4017: 9, 87911: 5, 105536: 8, 173679: 9, 203075: 8, 236340: 8, 24186: 5, 219683: 4, 68128: 8, 75149: 8, 228021: 8, 245295: 7, 273976: 4, 158226: 8, 32599: 9, 70541: 8, 81207: 9, 55490: 9, 134434: 7, 204676: 10, 161041: 8, 198243: 7, 169600: 7, 115488: 5, 76888: 7, 179327: 6, 53408: 7, 91281: 10, 68436: 9, 142672: 9, 80071: 7, 214213: 10, 243917: 6, 79525: 8, 67072: 1

{11676: 10, 85526: 10, 16795: 8, 198046: 8, 129716: 10, 269799: 9, 37712: 9, 70222: 9, 85043: 8, 87555: 7, 112765: 10, 114988: 7, 126296: 7, 139359: 8, 179826: 9, 189334: 10, 267510: 7, 28222: 9, 63625: 10, 128325: 8, 196053: 6, 239584: 10, 136252: 10, 31226: 6, 188010: 6, 262998: 8, 270605: 5, 43486: 9, 111174: 10, 174615: 6, 112512: 10, 152579: 8, 245295: 9, 64241: 10, 91184: 7, 234359: 6, 32275: 8, 187574: 8, 33818: 6, 26883: 9, 217121: 5, 127131: 9, 172613: 10, 225199: 5, 116183: 7, 174596: 7, 197206: 7, 258588: 3, 273473: 7, 262666: 10, 56323: 10, 124747: 9, 221753: 8, 243328: 8, 82942: 7, 5091: 8, 166646: 9, 232740: 9, 244823: 10, 41985: 8, 126149: 7, 84866: 5, 50845: 10, 186762: 7, 58834: 6, 61348: 8, 176953: 10, 231638: 7, 236621: 9, 1427: 10, 2295: 6, 2414: 7, 8874: 5, 27839: 10, 34428: 3, 35797: 8, 37548: 10, 43714: 6, 48622: 5, 72101: 9, 80477: 10, 82356: 10, 86692: 10, 86913: 8, 127801: 7, 130320: 8, 132954: 10, 133733: 8, 137994: 9, 172124: 6, 173253: 6, 174929: 10, 202273

{13552: 9, 27684: 8, 105979: 7, 153718: 7, 217375: 8, 229011: 7, 232131: 6, 260897: 7, 261829: 5, 263163: 7, 124874: 7, 156150: 10, 178667: 6, 221232: 8, 60244: 8, 21484: 6, 36003: 10, 43842: 5, 80820: 8, 87555: 8, 142524: 8, 179730: 10, 227447: 5, 138543: 7, 93426: 8, 140662: 1, 199041: 10, 6575: 4, 30735: 4, 94347: 7, 143517: 8, 116499: 3, 43163: 8, 256843: 8, 57200: 8, 71752: 9, 26487: 10, 125287: 8, 243607: 10, 254299: 6, 262525: 8, 141719: 8, 120333: 7, 26377: 8, 92798: 7, 234399: 9, 55799: 7, 162784: 7, 264709: 8, 143809: 6, 176576: 6, 186032: 9, 102945: 8, 130705: 10, 260388: 10, 105771: 10, 128132: 9, 273440: 10, 102668: 8, 95210: 8, 134478: 9, 252874: 7, 7084: 10, 266410: 3, 112402: 7, 91282: 9, 90279: 7, 110662: 8, 113560: 8, 90297: 10, 44706: 8, 100929: 8, 155060: 8, 157136: 3, 175066: 7, 197462: 8, 206372: 9, 208698: 6}
{11676: 5, 22343: 8, 23902: 9, 41841: 8, 44728: 7, 137190: 7, 140080: 9, 89602: 8, 149908: 8, 245295: 7, 131402: 8, 167521: 7, 49155: 8, 205383: 7, 202564: 

{7346: 8, 115003: 7, 82407: 4, 153662: 10, 204864: 10, 269799: 9, 38023: 8, 69971: 8, 95359: 8, 211426: 8, 259629: 7, 8253: 10, 163326: 9, 130571: 9, 169663: 5, 196466: 8, 245645: 10, 152645: 8, 91113: 9, 244814: 10, 136348: 8, 26628: 9, 44837: 8, 149907: 9, 15945: 8, 49212: 4, 73716: 9, 74413: 8, 240663: 10, 16659: 8, 29429: 9, 86296: 6, 48007: 8, 34702: 7, 138120: 7, 154910: 9, 168959: 7, 183518: 9, 253827: 8, 275094: 8}
{11676: 8, 7346: 7, 157273: 8, 60244: 9, 38556: 5, 76151: 10, 62272: 9, 46909: 9, 169057: 9, 69156: 9, 242775: 10, 16510: 9, 240235: 9, 249650: 9}
{11676: 7, 46398: 10, 236322: 8, 48268: 7, 114544: 8, 246655: 5, 6242: 8, 36907: 8, 15418: 10, 95359: 10, 69078: 9, 132624: 5, 8066: 9, 110934: 9, 6575: 7, 20180: 8, 223087: 5, 215278: 3, 43486: 6, 83971: 9, 227282: 7, 246507: 5, 181687: 6, 1131: 8, 174072: 8, 180042: 7, 189973: 3, 206563: 10, 1548: 8, 194425: 4, 128119: 10, 63004: 7, 98804: 7, 215988: 5, 9141: 9, 127755: 10, 183952: 5, 127893: 6, 53069: 5, 96602: 8, 23948

...

{11676: 8, 78973: 7, 102647: 10, 135149: 6, 232131: 6, 250101: 10, 261829: 6, 204864: 9, 19085: 9, 48046: 9, 178199: 6, 242639: 2, 268032: 9, 234828: 10, 225595: 10, 224997: 4, 268932: 8, 204783: 7, 5998: 8, 82955: 9, 144318: 8, 154443: 9, 121170: 5, 258152: 5, 124983: 8, 132282: 7, 128014: 7, 32599: 7, 182838: 10, 19659: 10, 139706: 9, 16730: 5, 76888: 9, 25919: 9, 257493: 5, 195861: 8, 82825: 8, 253311: 6, 175278: 6, 53365: 9, 72470: 4, 229707: 5, 199571: 8, 10221: 7, 17505: 1, 31266: 4, 75269: 9, 182238: 8, 216725: 5, 223224: 8, 241267: 8}
{214138: 8}
{174861: 8}
{11676: 9, 258534: 8, 95359: 9, 221543: 4, 253005: 8, 18155: 7, 191671: 9, 258422: 10, 45615: 6, 173225: 9, 18022: 8, 55567: 5}
{11676: 8, 11224: 7, 59038: 10, 75819: 10, 105937: 8, 224764: 8, 277157: 7, 7125: 4, 21031: 8, 39773: 10, 51365: 10, 61901: 9, 155580: 9, 158385: 8, 201677: 8, 3509: 8, 21395: 7, 23619: 5, 23637: 9, 31252: 7, 35148: 6, 42309: 4, 46159: 8, 48928: 6, 74511: 9, 79269: 2, 95011: 9, 111468: 8, 142387: 9

{11676: 3, 92861: 6, 36606: 5, 81492: 5, 94494: 5, 133747: 4, 127168: 7, 85757: 5, 206944: 6, 210587: 5, 125928: 5, 244593: 5, 244964: 5, 225066: 8, 63294: 3, 8245: 8, 175828: 6, 80516: 5, 200643: 10, 104808: 2, 138379: 7, 247240: 1, 55493: 9, 30324: 8, 177233: 8, 198564: 4, 224221: 5, 119541: 8, 102955: 8, 228313: 9, 231395: 1, 7890: 8, 2499: 5, 7404: 6, 16533: 2, 32184: 5, 33004: 4, 41897: 4, 49001: 5, 60191: 5, 74443: 7, 101616: 7, 103824: 6, 107109: 7, 109372: 5, 121317: 7, 162629: 5, 163871: 3, 202553: 5, 204044: 4, 213535: 5, 262033: 2, 274339: 5, 275841: 6}
{7346: 8, 115003: 8, 204864: 8, 244171: 10, 271622: 8, 32440: 7, 210485: 8, 235543: 10, 218568: 7, 110934: 10, 70402: 9, 177375: 8, 196466: 6, 13273: 9, 29209: 10, 266807: 7, 241495: 8, 165308: 9, 55655: 9, 60337: 9, 188490: 10, 182969: 7, 27875: 8, 103336: 8, 219219: 7, 107951: 8, 118228: 8, 138761: 9, 160180: 7, 183952: 8, 123115: 7, 218497: 8, 278194: 6, 151928: 10, 143388: 5, 34480: 10, 50066: 10, 50961: 8, 86759: 5, 9365

{189835: 5}
{153662: 8, 129465: 7, 61854: 8}
{35859: 7}
{78553: 10}
{11676: 10, 252865: 9, 50895: 7, 98391: 9, 157247: 2, 169309: 7, 36673: 6, 54335: 8, 237990: 5, 129469: 4, 23965: 10, 79182: 6, 109014: 8, 208790: 3}
{189835: 5, 272225: 9}
{189835: 5}
{32599: 8}
{189835: 5}
{185011: 9}
{136205: 1, 87051: 9, 13273: 9, 149908: 8, 177590: 10, 136255: 7, 152061: 10, 123080: 10, 136704: 5, 231972: 8}
{11676: 9, 177458: 5, 16795: 9, 35859: 9, 76352: 8, 93363: 9, 136396: 8, 174642: 8, 241980: 10, 273979: 7, 34588: 2, 148744: 8, 153662: 9, 204864: 6, 60244: 5, 15049: 8, 43842: 8, 55178: 7, 81848: 7, 114444: 5, 136747: 10, 140000: 8, 141493: 10, 150124: 10, 162639: 10, 170513: 7, 174216: 8, 208815: 10, 208829: 7, 219546: 7, 224525: 10, 256402: 7, 266056: 9, 269890: 7, 87141: 8, 274423: 6, 163430: 10, 226545: 10, 272385: 10, 131154: 8, 141491: 7, 47316: 10, 16916: 10, 176214: 8, 117384: 10, 223190: 9, 208671: 10, 230496: 9, 245645: 7, 56447: 6, 177072: 7, 221687: 10, 96357: 9, 183131: 10, 32122

{11676: 8, 110912: 10, 113270: 8, 115003: 9, 213760: 10, 221445: 9, 231237: 10, 235935: 8, 6543: 7, 7125: 10, 19974: 9, 21576: 6, 32721: 8, 37974: 10, 43246: 9, 53042: 10, 64436: 7, 69971: 8, 88693: 9, 89361: 10, 95359: 9, 98787: 9, 110029: 10, 114988: 7, 137190: 7, 141710: 7, 142600: 10, 145165: 8, 145449: 10, 171118: 7, 206305: 9, 210485: 9, 229329: 8, 251843: 8, 263460: 7, 70594: 10, 62542: 10, 89602: 8, 95991: 8, 213126: 9, 218568: 9, 243891: 6, 46443: 8, 110934: 9, 128325: 8, 168144: 10, 6575: 9, 95173: 8, 162780: 8, 177072: 8, 25966: 8, 86202: 7, 135351: 9, 215421: 8, 135703: 10, 136326: 9, 75149: 7, 20703: 8, 88793: 10, 66789: 5, 160571: 8, 58342: 9, 91058: 9, 200493: 9, 37584: 8, 256603: 9, 88652: 10, 136348: 8, 253771: 10, 19573: 9, 30316: 5, 54035: 8, 123567: 8, 17906: 7, 122867: 10, 256507: 7, 160298: 8, 277623: 8, 74377: 7, 4999: 10, 87114: 10, 132229: 7, 233952: 8, 242763: 7, 252783: 9, 270728: 8, 6115: 10, 163545: 10, 101091: 10, 148548: 8, 174031: 9, 182159: 6, 213837: 7

{63714: 10}
{152186: 8, 166922: 9, 224646: 10, 125878: 10, 33818: 8, 241051: 10, 171846: 9, 138124: 5, 147851: 6, 154954: 7, 102607: 9, 157154: 8}
{212088: 10}
{63714: 10}
{25981: 5}
{56271: 10, 76626: 8, 144516: 7, 38818: 7, 24835: 9, 170170: 7, 212191: 4, 214194: 9}
{218360: 7}
{227538: 8}
{46398: 6, 388: 7, 125519: 10, 216683: 3, 36907: 10, 105517: 2, 157273: 8, 10118: 10, 30261: 9, 39281: 8, 62862: 8, 93257: 3, 117111: 10, 132663: 9, 179772: 10, 245978: 8, 268030: 7, 237856: 10, 61888: 10, 106849: 10, 163804: 3, 221257: 7, 97874: 10, 97754: 6, 225078: 6, 148258: 7, 14638: 6, 128045: 8, 243312: 3, 28467: 10, 253310: 7, 244572: 10, 94455: 2, 30319: 9, 31141: 10, 38457: 8, 65465: 8, 76480: 7, 90752: 8, 92798: 5, 93358: 6, 99216: 10, 116582: 10, 132687: 8, 138999: 6, 141840: 7, 143663: 10, 149245: 5, 170151: 6, 175851: 10, 179885: 10, 183129: 10, 189718: 8, 193272: 6, 213859: 10, 221753: 5, 227686: 1, 243376: 9, 244340: 8, 250674: 6, 252917: 8, 275154: 8}
{22625: 9, 34588: 10, 153662: 

In [55]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    groupDF= groupDF.reset_index(drop = True)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)
print(vector)
#pairwiseSimilarity = cosine_similarity(vector)

  (0, 976)	9.0
  (0, 2003)	9.0
  (1, 958)	8.0
  (1, 6481)	10.0
  (1, 7262)	8.0
  (1, 11796)	9.0
  (1, 17042)	9.0
  (1, 17431)	8.0
  (1, 17829)	8.0
  (1, 17937)	8.0
  (1, 19192)	5.0
  (1, 20976)	9.0
  (2, 66)	9.0
  (2, 958)	8.0
  (2, 1330)	10.0
  (2, 2094)	9.0
  (2, 2495)	9.0
  (2, 2902)	8.0
  (2, 3565)	6.0
  (2, 3991)	6.0
  (2, 4113)	10.0
  (2, 5249)	5.0
  (2, 5818)	10.0
  (2, 6264)	5.0
  (2, 6327)	3.0
  :	:
  (1852, 1325)	5.0
  (1853, 14231)	9.0
  (1854, 10622)	8.0
  (1854, 14683)	10.0
  (1855, 386)	6.0
  (1855, 17575)	6.0
  (1856, 3479)	9.0
  (1857, 4632)	6.0
  (1858, 7077)	8.0
  (1859, 11585)	7.0
  (1860, 14289)	9.0
  (1861, 7196)	10.0
  (1861, 10722)	9.0
  (1861, 12685)	7.0
  (1862, 6628)	9.0
  (1862, 24480)	10.0
  (1863, 6449)	8.0
  (1864, 7231)	1.0
  (1865, 9269)	7.0
  (1865, 14802)	8.0
  (1866, 9921)	10.0
  (1867, 18070)	4.0
  (1868, 21539)	9.0
  (1869, 5297)	7.0
  (1870, 19707)	7.0


In [56]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)
vector1 = dictVectorizer.inverse_transform(vector)
print(vector1)
pairwiseSimilarity = cosine_similarity(vector)

[{11944: 9.0, 23902: 9.0}, {11676: 8.0, 71837: 10.0, 81274: 8.0, 131398: 9.0, 191521: 9.0, 196118: 8.0, 201537: 8.0, 202664: 8.0, 216420: 5.0, 236173: 9.0}, {901: 9.0, 11676: 8.0, 16383: 10.0, 24922: 9.0, 28993: 9.0, 32440: 8.0, 39305: 6.0, 43992: 6.0, 45305: 10.0, 57974: 5.0, 64171: 10.0, 69405: 5.0, 70052: 3.0, 73815: 8.0, 76350: 7.0, 77228: 8.0, 81227: 9.0, 81855: 8.0, 86243: 8.0, 114279: 9.0, 120080: 10.0, 126156: 8.0, 140000: 6.0, 147156: 8.0, 147162: 9.0, 148258: 7.0, 149908: 8.0, 154112: 9.0, 159863: 8.0, 164096: 7.0, 168646: 9.0, 176638: 8.0, 219181: 7.0, 221672: 8.0, 231613: 8.0, 231854: 6.0, 234554: 10.0, 235948: 7.0, 246678: 2.0, 249626: 8.0, 256820: 10.0, 262407: 7.0, 262902: 9.0}, {3154: 8.0, 8019: 9.0, 10030: 7.0, 10351: 7.0, 11676: 6.0, 12538: 10.0, 16996: 8.0, 38248: 10.0, 41084: 7.0, 41763: 8.0, 49809: 6.0, 50216: 10.0, 50526: 10.0, 52584: 10.0, 53228: 10.0, 64027: 7.0, 70129: 7.0, 70554: 5.0, 71712: 6.0, 80538: 7.0, 86243: 7.0, 89551: 10.0, 92215: 7.0, 93929: 9.0, 103

In [57]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)
#print(vector)
pairwiseSimilarity = cosine_similarity(vector)
print(pairwiseSimilarity)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.04673291 ... 0.         0.         0.        ]
 [0.         0.04673291 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [62]:
def printBookDetails(bookID):
    print(dataset1[dataset1['ISBN']==bookID]['Book-Title'].values[0])
    

def getTopRecommandations(bookID):
    collaborative = []
    row = reverseIndexMap[bookID]
    print("Input Book:")
    printBookDetails(bookID)
    
    print("\nRECOMMENDATIONS:\n")
    
    mn = 0
    similar = []
    for i in np.argsort(pairwiseSimilarity[row])[::-1]:
          if dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0] not in similar:
               
                if mn>=10:
                      break
                mn+=1
                similar.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                printBookDetails(indexMap[i])
                collaborative.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                #collaborative.append(pairwiseSimilarity[i][np.argsort(pairwiseSimilarity[i])[::-1][0]])
            
    return collaborative

In [63]:
k = list(dataset1['Book-Title'])
m = list(dataset1['ISBN'])

collaborative = getTopRecommandations(m[k.index(bookName)])
print(collaborative)

Input Book:
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))

RECOMMENDATIONS:

Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
Fried Green Tomatoes at the Whistle Stop Cafe
The Two Towers (The Lord of the Rings, Part 2)
Cold Sassy Tree
The Secret Garden
Anne of Green Gables (Anne of Green Gables Novels (Paperback))
["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))", 'Harry Potter and the Chamber of Secrets (Book 2)', 'Harry Potter and the Prisoner of Azkaban (Book 3)', 'Harry Potter and the Goblet of Fire (Book 4)', 'Harry Potter and the Order of the Phoenix (Book 5)', 'Fried Green Tomatoes at the Whistle Stop Cafe', 'The Two Towers (The Lord of the Rings, Part 2)', 'Cold Sassy Tree', 'The Secret Garden', 'Anne of Green Gables (Anne of Green Ga

In [ ]:
df = pd.DataFrame(dataset1['Book-Title'].value_counts())
df['Total-Ratings'] = df['Book-Title']
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
df = df.drop('index',axis=1)

df = dataset1.merge(df, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left') #df contains unique book details along with total ratings
df = df.drop(['Year-Of-Publication','Publisher','Age','City','State','Country'], axis=1)

popularity_threshold = 50
popular_book = df[df['Total-Ratings'] >= popularity_threshold]
popular_book = popular_book.reset_index(drop = True)


testdf = pd.DataFrame()
testdf['ISBN'] = popular_book['ISBN']
testdf['Book-Rating'] = popular_book['Book-Rating']
testdf['User-ID'] = popular_book['User-ID']
testdf = testdf[['User-ID','Book-Rating']].groupby(testdf['ISBN']) #rows=isbn,columns=users,testdf[isbn][users]=book-rating

listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)
pairwiseSimilarity = cosine_similarity(vector)

def printBookDetails(bookID):
    print(dataset1[dataset1['ISBN']==bookID]['Book-Title'].values[0])
    

def getTopRecommandations(bookID):
    collaborative = []
    row = reverseIndexMap[bookID]
    print("Input Book:")
    printBookDetails(bookID)
    
    print("\nRECOMMENDATIONS:\n")
    
    mn = 0
    similar = []
    for i in np.argsort(pairwiseSimilarity[row])[::-1]:
          if dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0] not in similar:
               
                if mn>=10:
                      break
                mn+=1
                similar.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                printBookDetails(indexMap[i])
                collaborative.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                #collaborative.append(pairwiseSimilarity[i][np.argsort(pairwiseSimilarity[i])[::-1][0]])
            
    return collaborative

In [64]:
print(dataset1['Book-Title'].values[0])
print(dataset1.head())

Clara Callan
         ISBN                                    Book-Title  \
0  0002005018                                  Clara Callan   
1  074322678X       Where You'll Find Me: And Other Stories   
2  0887841740                            The Middle Stories   
3  1552041778                                      Jane Doe   
4  1567407781  The Witchfinder (Amos Walker Mystery Series)   

            Book-Author  Year-Of-Publication                 Publisher  \
0  Richard Bruce Wright                 2001     HarperFlamingo Canada   
1           Ann Beattie                 2002                  Scribner   
2           Sheila Heti                 2004     House of Anansi Press   
3          R. J. Kaiser                 1999                Mira Books   
4     Loren D. Estleman                 1998  Brilliance Audio - Trade   

   User-ID  Book-Rating  Age  
0        8            5  NaN  
1        8            5  NaN  
2        8            5  NaN  
3        8            5  NaN  
4       

In [220]:
print(popular_book.head())

                                          Book-Title  Total-Ratings  \
0  Harry Potter and the Prisoner of Azkaban (Book 3)            277   
1       Harry Potter and the Goblet of Fire (Book 4)            247   
2  Harry Potter and the Order of the Phoenix (Boo...            211   
3     Harry Potter and the Sorcerer's Stone (Book 1)            176   
4     The Two Towers (The Lord of the Rings, Part 2)            136   

   Average Rating        ISBN     score  
0               9  0140126562  8.971768  
1               9  0452264464  8.968407  
2               9  0345465083  8.963141  
3               9  0380717018  8.956014  
4               9  0743470389  8.943535  


In [54]:
data = (dataset1.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().
        rename(columns = {'Book-Rating': 'Total-Rating'})[['Book-Title', 'Total-Rating']])

result = pd.merge(data, dataset1, on='Book-Title',)
result = result[result['Total-Rating'] >= popularity_threshold]
result = result.reset_index(drop = True)

matrix = result.pivot_table(index = 'Book-Title', columns = 'User-ID', values = 'Book-Rating').fillna(0)
up_matrix = csr_matrix(matrix)

model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(up_matrix)

bookName="Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"
distances, indices = model.kneighbors(matrix.loc[bookName].values.reshape(1, -1), n_neighbors = 5+1)
print("\nRecommended books:\n")
for i in range(0, len(distances.flatten())):
    if i > 0:
        print(matrix.index[indices.flatten()[i]])


Recommended books:

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
The Fellowship of the Ring (The Lord of the Rings, Part 1)


In [59]:
#print(data.head())
#print(dataset1.head())
print(matrix.head())

User-ID              16      26      32      42      51      91      97      \
Book-Title                                                                    
1984                    0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1st to Die: A Novel     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2nd Chance              0.0     0.0     0.0     0.0     0.0     0.0     0.0   
A Bend in the Road      0.0     0.0     0.0     0.0     0.0     0.0     0.0   
A Case of Need          0.0     0.0     0.0     0.0     0.0     0.0     0.0   

User-ID              99      114     125     ...  278755  278773  278798  \
Book-Title                                   ...                           
1984                    0.0     0.0     0.0  ...     0.0     0.0     0.0   
1st to Die: A Novel     0.0     0.0     0.0  ...     0.0     0.0     0.0   
2nd Chance              0.0     0.0     0.0  ...     0.0     0.0     0.0   
A Bend in the Road      0.0     0.0     0.0  ...     0.0     0.0  

In [60]:
print(up_matrix)

  (0, 17)	9.0
  (0, 120)	10.0
  (0, 155)	8.0
  (0, 158)	10.0
  (0, 198)	10.0
  (0, 494)	9.0
  (0, 508)	8.0
  (0, 600)	5.0
  (0, 679)	6.0
  (0, 775)	10.0
  (0, 1000)	9.0
  (0, 1122)	8.0
  (0, 1621)	7.0
  (0, 2068)	10.0
  (0, 2330)	8.0
  (0, 2339)	10.0
  (0, 2678)	7.0
  (0, 2691)	9.0
  (0, 2768)	9.0
  (0, 2799)	10.0
  (0, 3198)	10.0
  (0, 3370)	10.0
  (0, 3406)	10.0
  (0, 3483)	9.0
  (0, 3549)	9.0
  :	:
  (313, 15790)	3.0
  (313, 15794)	8.0
  (313, 15796)	7.0
  (313, 15905)	9.0
  (313, 15925)	7.0
  (313, 16051)	8.0
  (313, 16577)	10.0
  (313, 16930)	8.0
  (313, 16952)	5.0
  (313, 16976)	5.0
  (313, 17092)	6.0
  (313, 17381)	10.0
  (313, 17775)	6.0
  (313, 17930)	9.0
  (313, 17938)	8.0
  (313, 18263)	9.0
  (313, 18287)	5.0
  (313, 18569)	7.0
  (313, 18738)	7.0
  (313, 18782)	8.0
  (313, 18970)	8.0
  (313, 19284)	8.0
  (313, 19287)	9.0
  (313, 19402)	7.0
  (313, 19558)	9.0


In [55]:
#CONTENT BASED FILTERING
'''popularity_threshold = 80
popular_book = df[df['Total-Ratings'] >= popularity_threshold]
popular_book = popular_book.reset_index(drop = True)
popular_book.shape'''
tf = TfidfVectorizer(ngram_range=(1, 2), min_df = 1, stop_words='english')
tfidf_matrix = tf.fit_transform(popular_book['Book-Title'])
tfidf_matrix.shape

normalized_df = tfidf_matrix.astype(np.float64)
cosine_similarities = cosine_similarity(normalized_df, normalized_df)
cosine_similarities.shape

MemoryError: Unable to allocate 14.8 GiB for an array with shape (44619, 44619) and data type float64

In [250]:
print("Recommended Books:\n")
isbn = book_data.loc[book_data['Book-Title'] == bookName].reset_index(drop = True).iloc[0]['ISBN']
content = []

idx = popular_book.index[popular_book['ISBN'] == isbn].tolist()[0]
similar_indices = cosine_similarities[idx].argsort()[::-1]
similar_items = []
for i in similar_indices:
    if popular_book['Book-Title'][i] != bookName and popular_book['Book-Title'][i] not in similar_items and len(similar_items) < number:
        similar_items.append(popular_book['Book-Title'][i])
        content.append(popular_book['Book-Title'][i])

for book in similar_items:
    print(book)

Recommended Books:



IndexError: list index out of range

In [ ]:
https://www.youtube.com/watch?v=OM4aZJW_Ojs
from Amit Kumar Rai to everyone:    5:17 PM
https://www.youtube.com/watch?v=MPQSj7fEdlA